# Woodwork Typing in Featuretools

Featuretools relies on having consistent typing across the creation of EntitySets, Primitives, Features, and feature matrices. Previously, Featuretools used its own type system that contained objects called Variables. Now and moving forward, Featuretools will use an external package for its typing: [Woodwork](https://woodwork.alteryx.com/en/stable/index.html).

Woodwork is a library that helps with the data typing of 2-dimensional tabular data structures. Initializing Woodwork on a DataFrame creates a namespace, `ww`, that contains physical, logical, and semantic data types that can be used within the Alteryx Open Source ecosystem as well as in general machine learning applications.

Understanding these Woodwork types and how they work together to create Woodwork's type system will allow users to build EntitySets that best represent their data, understand the possible input and return types for Featuretools' Primitives, and understand what features will get generated from a given set of data and primitives.

- Physical Type: defines how the data is stored on disk or in memory.
- Logical Type: defines how the data should be parsed or interpreted.
- Semantic Tag(s): provides additional data about the meaning of the data or how it should be used.

Read the [Understanding Woodwork Logical Types and Semantic Tags](link) guide for an in-depth walkthrough of the available Woodwork types.

Read the [Working with Woodwork Types and Tags](link) guide to learn more about manipulating these types on a Woodwork DataFrame.

For users that are familiar with the old `Variable` ojbects, the [Transitioning from Variables to Woodwork](link) guide will be useful for converting Variable types to Woodwork types.

## Physical Types 
Physcial types define how the data in a Woodwork DataFrame is stored on disk or in memory. You might also see the physical type for a column referred to as the column’s `dtype`.

Pandas, Dask, and Koalas DataFrames rely on these dtypes when performing DataFrame operations, so knowing a Woodwork table's physical types is important for any DataFrame operations that might be performed. Each `LogicalType` class has a single physical type associated with it, though multiple logical types can have the same physical type.

## Logical Types

Multiple logical types may have the same physical types, because a logical type adds additional information about how data should be interpreted or parsed beyond what can be contained in a physical type. 

For example, email addresses and phone numbers would typically both be stored in a data column with a physical type of `string`. However, when reading, validating, and using these two types of information, different rules apply. For email addresses, the presence of the `@` symbol is important. For phone numbers, you might want to confirm that only a certain number of digits are present or use the first three digits to determine an area-code, and special characters might be restricted to `+`, `-`, `(` or `)`. In this particular example Woodwork defines two different logical types to separate these parsing needs: `EmailAddress` and `PhoneNumber`.

Woodwork provides many different logical types, which can be seen with the `list_logical_types` function.

In [74]:
import featuretools as ft
ft.list_logical_types()

,name,type_string,description,physical_type,standard_tags,is_default_type,is_registered,parent_type
0,Address,address,Represents Logical Types that contain address ...,string,{},True,True,None
1,Age,age,Represents Logical Types that contain non-nega...,int64,{numeric},True,True,Integer
2,AgeNullable,age_nullable,Represents Logical Types that contain non-nega...,Int64,{numeric},True,True,IntegerNullable
3,Boolean,boolean,Represents Logical Types that contain binary v...,bool,{},True,True,BooleanNullable
4,BooleanNullable,boolean_nullable,Represents Logical Types that contain binary v...,boolean,{},True,True,None
5,Categorical,categorical,Represents Logical Types that contain unordere...,category,{category},True,True,None
6,CountryCode,country_code,Represents Logical Types that contain categori...,category,{category},True,True,Categorical
7,Datetime,datetime,Represents Logical Types that contain date and...,datetime64[ns],{},True,True,None
8,Double,double,Represents Logical Types that contain positive...,float64,{numeric},True,True,None
9,EmailAddress,email_address,Represents Logical Types that contain email ad...,string,{},True,True,None


Featuretools will perform type inference to assign logical types to the data in EntitySets, but it is also possible to specify which logical types should be set for any column (provided that the data in that column is compatible with the logical type). 

To learn more about how logical types are used in EntitySets, see the [Creating EntitySets](link) guide. To learn more about setting logical types directly on a DataFrame, see the Woodwork guide on [working with Logical Types](https://woodwork.alteryx.com/en/stable/guides/understanding_types_and_tags.html#Working-with-Logical-Types). 

## Semantic Tags
The `standard_tags` column in the `list_logical_types` output above contains semantic tags that Woodwork applies to specific logical types. These tags provide even more context about the meaning and potential uses of a column. 

One use of semantic tags can be in grouping certain columns toghether. For example: `Age` and `Integer` have the `int64` physical type, and `Double` has the `float64` physical type, but all three have the `numeric` tag, which ties them together in a way that physical types and logical types are not able to do alone. 

Semantic tags can also be used to provide further specificity beyond logical types. Consider a dataset with 2 date columns: a signup date and a user birth date. Both of these columns have the same physical type (`datetime64[ns]`), and both have the same logical type (`Datetime`), but the way one might interpret or use these columns is not the same. This time, semantic tags can be used to differentiate these columns. For example, you might want to add the `date_of_birth` semantic tag to the user birth date column to indicate this column has special meaning and could be used to compute a user’s age. Computing an age from the signup date column would not make sense, so the semantic tag can be used to differentiate between what the dates in these columns mean.

### Woodwork Standard Tags

Woodwork will add certain tags to columns at initialization. As we mentioned above, one type of these tags is standard tags that may be associated with different sets logical types that fall under certain predefined categories.

The standard tags for each logical type can be seen in the `standard_tags` column of the `list_logical_types` output above. 

To learn more about working with Woodwork's standard semantic tags directly on a DataFrame, see the Woodwork guide on [working with semantic tags](https://woodwork.alteryx.com/en/stable/guides/understanding_types_and_tags.html#Standard-Tags).

There are also 2 tags that Woodwork adds only to index columns. If no index columns have been specified, these tags are not present:

* `'index'` - on the index column, when specified
* `'time_index'` on the time index column, when specified

These `index` and `time_index` tags, which have special meaning, can be controlled by the user at EntitySet creation or through Woodwork methods directly on the DataFrame. This is discussed in more detail in the Woodwork guide on [setting the index](https://woodwork.alteryx.com/en/stable/guides/understanding_types_and_tags.html#Setting-the-index).

To get a list of the standard, index, and time index tags, you can use the `list_semantic_tags` function.

In [2]:
ft.list_semantic_tags()

,name,is_standard_tag,valid_logical_types
0,numeric,True,"[Age, AgeNullable, Double, Integer, IntegerNul..."
1,category,True,"[Categorical, CountryCode, Ordinal, PostalCode..."
2,index,False,"[Integer, Double, Categorical, Datetime]"
3,time_index,False,[Datetime]
4,date_of_birth,False,[Datetime]


Above we see the standard tags that are defined within Woodwork. These tags inform how Featuretools is able to interpret data, but in order to generate the full set of features, Featuretools must create a few tags of its own.

### Semantic Tags in Featuretools

Just like Woodwork specifies semantic tags internally, Featuretools also has certain semantic tags that have a specific meaning when they are present on a column.

- `'last_time_index'` - added by Featuretools to the last time index column of a DataFrame. Indicates that this column has been created by Featuretools.
- `'foreign_key'` - used to indicate that a column has a parent column that's a primary key of another dataframe in the EntitySet. Is added by Featuretools when a Relationship is created between two DataFrames.
- `'date_of_birth'` - indicates that a column can be used for Primitive operations that require a date of birth. Must be added at EntitySet creation to be present.


## Woodwork Throughout Featuretools

Now that we've described the elements that make up Woodwork's type system, lets see them in action in Featuretools.

### Woodwork in EntitySets
For more information on building EntitySets using Woodwork, see the [EntitySet guide](link). Let's look at the Woodwork typing information as it's stored in a demo EntitySet of flight data:

In [38]:
es = ft.demo.load_retail()

es

Entityset: demo_retail_data
  DataFrames:
    order_products [Rows: 401604, Columns: 8]
    products [Rows: 3684, Columns: 4]
    orders [Rows: 22190, Columns: 6]
    customers [Rows: 4372, Columns: 3]
  Relationships:
    order_products.product_id -> products.product_id
    order_products.order_id -> orders.order_id
    orders.customer_name -> customers.customer_name

The EntitySet representation above contains no Woodwork typing information, as it is a representation of a relational dataset, and Woodwork currently supports single tables. To look at the Woodwork typing information, we first select a single DataFrame from the EntitySet, and then access the Woodwork information via the `ww` namespace:

In [55]:
df = es['products']
df.head()

,product_id,description,first_order_products_time,_ft_last_time
85123A,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,2011-12-09 11:34:00
71053,71053,WHITE METAL LANTERN,2010-12-01 08:26:00,2011-12-07 14:12:00
84406B,84406B,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,2011-12-05 14:30:00
84029G,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,2011-12-09 11:26:00
84029E,84029E,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,2011-12-09 09:07:00


In [56]:
df.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
product_id,category,Categorical,['index']
description,string,NaturalLanguage,[]
first_order_products_time,datetime64[ns],Datetime,['time_index']
_ft_last_time,datetime64[ns],Datetime,['last_time_index']


Notice how the three columns provided to show this DataFrame's typing information are the three types of typing information outlined at the beginning of this guide. To reiterate: By defining physcial types, logical types, and semantic tags for each column in a DataFrame, we've defined a DataFrame's Woodwork schema, and with it, we can gain an understanding of the contents of each column.

This column-specific typing information that exists for every column in every table in an EntitySet is an integral part of Deep Feature Synthesis' ability to generate features for an EntitySet.

### Woodwork in DFS
As the units of computation in Featuretools, Primitives need to be able to specify the input types that they allow as well as have a predictable return type. For an in-depth explanation of Primitives in Featuretools, see the [Feature Primitives](link) guide. Here, we'll look at how the Woodwork types come together into a `ColumnSchema` object to describe Primitive input and return types.

Below is a Woodwork `ColumnSchema` that we've obtained from the `'product_id'` column in the `products` DataFrame in the retail EntitySet.

In [45]:
products_df = es['products']
product_ids_series = products_df.ww['product_id']
column_schema = product_ids_series.ww.schema
column_schema

<ColumnSchema (Logical Type = Categorical) (Semantic Tags = ['index'])>

This combination of logical type and semantic tag typing information is a `ColumnSchema`. In the case above, the `ColumnSchema` describes the **type definition** for a single column of data.

However, it's important to note that there is no physical type in a `ColumnSchema`. This is because a `ColumnSchema` is a collection of Woodwork types that doesn't have any data tied to it and therefore has no physical representation of any data. 

Because a `ColumnSchema` object is not tied to any data, it can also be used to describe a **type space** into which other columns may or may not fall.

This flexibility of the `ColumnSchema` class means that each column in an EntitySet ultimately can be described by a type definition `ColumnSchema`, and each Primitive defines the input and return types using a `ColumnSchema` type space.

Let's look at a different column in a different DataFrame to see how this works:

In [58]:
order_products_df = es['order_products']
order_products_df.head() 

,order_product_id,order_id,product_id,quantity,order_date,unit_price,total,_ft_last_time
0,0,536365,85123A,6,2010-12-01 08:26:00,4.2075,25.245,2010-12-01 08:26:00
1,1,536365,71053,6,2010-12-01 08:26:00,5.5935,33.561,2010-12-01 08:26:00
2,2,536365,84406B,8,2010-12-01 08:26:00,4.5375,36.300,2010-12-01 08:26:00
3,3,536365,84029G,6,2010-12-01 08:26:00,5.5935,33.561,2010-12-01 08:26:00
4,4,536365,84029E,6,2010-12-01 08:26:00,5.5935,33.561,2010-12-01 08:26:00


In [46]:
quantity_series = order_products_df.ww['quantity']
column_schema = quantity_series.ww.schema
column_schema

<ColumnSchema (Logical Type = Integer) (Semantic Tags = ['numeric'])>

The `ColumnSchema` above has been pulled from the `'quantity'` column in the `order_products` table in the retail EntitySet. This is a **type definition**. 

If we look at the `order_products` table's Woodwork types below, we can see that there are several columns that will have similar `ColumnSchema` type definitions. If we wanted to describe subsets of those columns, we could define several `ColumnSchema` **type spaces**

In [59]:
es['order_products'].ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
order_product_id,int64,Integer,['index']
order_id,category,Categorical,"['foreign_key', 'category']"
product_id,category,Categorical,"['foreign_key', 'category']"
quantity,int64,Integer,['numeric']
order_date,datetime64[ns],Datetime,['time_index']
unit_price,float64,Double,['numeric']
total,float64,Double,['numeric']
_ft_last_time,datetime64[ns],Datetime,['last_time_index']


Below are several `ColumnSchema`s that all would include our `quantity` column, but each of them describes a different type space. These `ColumnSchema`s get more restrictive as we go down:

##### Entire Table
No restrictions have been placed; any column falls into this definition. This would include the whole DataFrame.

In [60]:
from woodwork.column_schema import ColumnSchema

ColumnSchema()

<ColumnSchema>

##### By Standard Tag
Only columns with the `numeric` tag apply. This can include Double, Integer, and Age logical type columns as well. It will not include the `index` column which, despite containing integers, has had its standard tags replaced by the `'index'` tag.

In [61]:
ColumnSchema(semantic_tags=['numeric'])

<ColumnSchema (Semantic Tags = ['numeric'])>

In [67]:
df = es['order_products'].ww.select(include='numeric')
df.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
quantity,int64,Integer,['numeric']
unit_price,float64,Double,['numeric']
total,float64,Double,['numeric']


##### By Logical Type
Only columns with logical type of `Double` are included in this definition. Does not require the `numeric` tag, so an index column (which has its standard tags removed) would still apply.

In [68]:
from woodwork.logical_types import Double

ColumnSchema(logical_type=Double)

<ColumnSchema (Logical Type = Double)>

In [69]:
df = es['order_products'].ww.select(include='Integer')
df.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
order_product_id,int64,Integer,['index']
quantity,int64,Integer,['numeric']


##### By Logical Type and Semantic Tag
The column must have logical type `Double` and have the `numeric` semantic tag, excluding index columns.

In [70]:
ColumnSchema(logical_type=Double, semantic_tags=['numeric'])

<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>

In [73]:
df = es['order_products'].ww.select(include='numeric')
df = df.ww.select(include='Integer')
df.ww

,Physical Type,Logical Type,Semantic Tag(s)
Column,,,
quantity,int64,Integer,['numeric']


In this way, a `ColumnSchema` can define a type space under which columns in a Woodwork DataFrame can fall. This is how Featuretools determines which columns in a DataFrame are valid for a Primitive in building Features during DFS.

Each Primitive has `input_types` and a `return_type` that are described by a Woodwork `ColumnSchema`. Every table in an EntitySet has Woodwork initialized on it. This means that when an EntitySet is passed into DFS, Featuretools can select the relevant columns in the DataFrame that are valid for the Primitive's `input_types`. We then get a Feature that has a `column_schema` property that indicates what that Feature's typing definition is in a way that lets DFS stack features on top of one another.

In this way, Featuretools is able to leverage the base unit of Woodwork typing information, the `ColumnSchema`, and use it in concert with an EntitySet of Woodwork DataFrames in order to build Features with Deep Feature Synthesis.